In [27]:
# Importing libraries
import pandas as pd
from datetime import datetime

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.mime.text import MIMEText

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import gspread_dataframe as gdf

In [28]:
# Defining the scope
scope = ["https://spreadsheets.google.com/feeds", 
         "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file", 
         "https://www.googleapis.com/auth/drive"]

# Adding credentials to the account
creds = (
    ServiceAccountCredentials
    .from_json_keyfile_name('keys\json_key.json', scope)
    )

# Authorizing the clientsheet
gc = (
    gspread
    .authorize(creds)
    )

# Getting the Sheet
data = (
    gc
    .open('job_application_helper')
    .worksheet('Details')
    )

# Reading data from the sheet
details = (
    gdf
    .get_as_dataframe(data)
    .dropna(how='all')
    )

In [29]:
# Function to send mails
def email(body, mail_address):
    # Defining parameters for login details
    sender_email = 'manujsinghwal2060@gmail.com'
    receiver_email = mail_address
    pass_key = 'pgyzwgapvmkknncf'

    # Composing email message
    msg = MIMEMultipart()
    msg['Subject'] = 'Application For Analytics Position (Test Mail)'
    msg['From'] = sender_email
    msg['To'] = receiver_email

    # Attaching body of the email
    msg.attach(MIMEText(body, 'html'))

    # Attaching resume as attachment
    attachment_path = 'docs/Resume.pdf'
    with open(attachment_path, 'rb') as attachment:
        part = MIMEApplication(attachment.read(), Name='Resume.pdf')
    part['Content-Disposition'] = f'attachment; filename="Resume.pdf"'
    msg.attach(part)

    # Connecting to the SMTP server and send the email
    try:
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.login(sender_email, pass_key)
        server.sendmail(sender_email, receiver_email, msg.as_string())
        print("Email sent successfully!")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        server.quit()

In [30]:
# Iterating over DataFrame
for i in range(0, len(details)):
    company_name = details.iloc[i]['company_name']
    mail_address = details.iloc[i]['mail_address']

    # Looping HTML summary template
    body = '''
    <p>Dear Hiring Manager,
    <br>
    <br>
    I hope this email finds you well. I am writing to express my interest in any openings for the Senior Data Analyst/Analytics Manager position at {0}.
    <br>
    <br>
    With a solid background in data analysis and extensive experience in utilizing Python, SQL, Power BI, and other related tools, I am excited about in any opportunity to contribute to your team and drive data-driven decision-making processes.
    <br>
    <br>
    In my previous roles, I have successfully led cross-functional teams in extracting actionable insights from complex datasets, resulting in improved business strategies and operational efficiencies. My proficiency in data visualization techniques has allowed me to effectively communicate findings to stakeholders at all levels, facilitating informed decision-making processes.
    <br>
    <br>
    <b>Some highlights of my qualifications include:</b>
    <br>
    1.  Proficiency in Python and SQL for data extraction, transformation, and analysis.
    <br>
    2.  Expertise in creating interactive dashboards and reports using Power BI, enabling stakeholders to gain insights at a glance.
    <br>
    3.  Proven track record of implementing machine learning algorithms to predict outcomes and optimize processes.
    <br>
    4.  Ensure seamless data integration by maintaining robust pipelines, automating ETL processes to extract, transform, and load data efficiently for accurate and timely analytics.
    <br>
    5.  Experience in leading and mentoring associates analysts, fostering a collaborative and growth-oriented team environment.
    <br>
    <br>
    <b>Some of my recent projects:</b>
    <br>
    1.  Payout Automation Model (<a href="https://github.com/manujsinghwal/automation-payout-model">Link</a>)
    <br>
    2.  Insights From Uber Supply And Demand Data (<a href="https://github.com/manujsinghwal/insights-from-uber-supply-and-demand-data/blob/main/insights_from_uber_supply_and_demand_data.ipynb">Link</a>)
    <br>
    3.  Uber Partner Business Modeling (<a href="https://github.com/manujsinghwal/uber-partner-business-modeling/blob/main/uber_partner_business_modeling.ipynb">Link</a>)
    <br>
    4.  Data Transformation From JSON To Tables (<a href="https://github.com/manujsinghwal/chama-data-transformation/blob/main/chama_data_transformation.ipynb">Link</a>)
    <br>
    5.  Website Traffic Analysis (<a href="https://github.com/manujsinghwal/website-traffic-analysis/blob/main/website_traffic_analysis.ipynb">Link</a>)
    <br>
    <br>
    I am particularly drawn to {0}'s commitment to leveraging data analytics to drive innovation and achieve business objectives. I am eager to bring my skills and expertise to your esteemed organization and contribute to its continued success.
    <br>
    <br>
    Thank you for considering my application. I have attached my resume for your review. I am available for an interview at your earliest convenience and look forward to the opportunity to discuss how my background, skills, and passion for data analytics align with the needs of {0}.
    <br>
    <br>
    Warm regards,
    <br>
    Manu Singhwal
    </p>
    <br>
    '''.format(company_name)

    email(body, mail_address)

Email sent successfully!
Email sent successfully!


In [31]:
# Clear sheet for updates
data.clear()

# Updating dataframe for sheet
current_timestamp = datetime.now()
details['n_times'] = details['n_times'].astype(int) + 1
details['last_sent_timestamp'] = current_timestamp

# Adding updated data to the sheet
gdf.set_with_dataframe(data, details)